Import potrebnih knjižnic za analizo podatkov:

In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Prebiranje datotek s podakti, tukaj se ignorira vrstice, ki pokvarijo strukturo podatkov oziroma v originalni datoteki niso del podatkov temvec prazna vrstica ipd. 
V programski kodi so seveda zamaknjene za -1.

Izberemo tudi samo stolpce z izbranimi točkami:  
A - datumi  
B - povprecje  
C:IW - izbrane tocke  


od IW naprej so pa stolpci prazni in nekje v koncu dokumenta spet nekatere vrednosti, ki niso pomembne

Stolpec z datumi se spremeni v datume.

In [65]:
file_path = 'podatki/Rizana_Zaledje_INCA_dnev_2020_2021_N1.xlsx'
df_padavine = pd.read_excel(file_path, sheet_name='Izbrane', skiprows=[0, 1, 3, 4], usecols="A,C:IW")

df_padavine["Datum"] = pd.to_datetime(df_padavine["Datum"], format="%m/%d/%y")
df_padavine = df_padavine.set_index('Datum')

preberemo se za pretok, po stoplcih:  
A - datum  
B - pretok  
C - prazen  
D - temp. vode  
E - mot. vode  

se preimenujem pretok da bo lazje obdelovati

In [64]:
file_path1 = 'podatki/Pretok_2020_2021_N1.xlsx'
df_pretok = pd.read_excel(file_path1, sheet_name='meritve', usecols="A, B")
df_pretok.rename(columns={"pretok (m3/s)": "pretok"}, inplace=True)

df_pretok["Datum"] = pd.to_datetime(df_pretok["Datum"], format="%d.%m.%Y")
df_pretok = df_pretok.set_index('Datum')


zdrzuimo oba dataframa s concat (axis = 1 pomeni da po stolpcih)

#### Normalne vrednosti in % odstopanja
- pogledati je treba katere so normalne vrednosti in kakšen procent predstavlja znatno odstopanje

In [71]:
df = pd.concat([df_pretok, df_padavine], axis=1)

print(f"Velikost: {df.shape}")
#print(f"Stolpci: {df.columns.tolist()}")
"""
x = 5
print(df_pretok.head(x))
print(df_padavine.head(x))
print(df.sample(x))
"""
print(f"Toliko vrstic vsebuje Nan: {df.isna().any(axis=1).sum()}")

#zaokroženo na 3 mesta kot podatki
avgPretok = df['pretok'].mean().round(3) 
modPretok = df["pretok"].mode().iloc[0] 
#mode vrne vecje stevilo zato treba z iloc preveriti prvega ne glede na index
medPretok = df["pretok"].median().round(3)

print(f"Povprečni pretok: {avgPretok}")
print(f"Mediana pretok: {medPretok}")
print(f"Najbolj pogost pretok: {modPretok}")

Velikost: (731, 256)
Toliko vrstic vsebuje Nan: 0
Povprečni pretok: 2.757
Mediana pretok: 0.909
Najbolj pogost pretok: 0.193
